# Location Recommendation for Office or Retail Spaces

## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis


import json # library to handle JSON files

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Done!")

Done!


In [2]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print("Done!")

Done!


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("Done")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

## Data Preparation

### Importing Dataset

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f352207a9645412fb96dae66feabee02 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OckQvsf4cvungExkZGTe2bXyQORnHKOkA3uW5R2iJEEr',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_f352207a9645412fb96dae66feabee02.get_object(Bucket='myproject-donotdelete-pr-ac7ejsbf9q4hjx',Key='Dataset_Rentals.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Street,Retail,Office
0,AIRPORT BOULEVARD,326.27,NaN
1,ALBERT STREET,89.32,52.02
2,ALEXANDRA ROAD,109.37,68.89
3,ANG MO KIO AVENUE 3,268.37,NaN
4,ANSON ROAD,82.41,65.91


### Data Cleaning

In [6]:
# Replace NaN values with 0
df_data_1['Office'].fillna(0, inplace = True) 
df_data_1['Retail'].fillna(0, inplace = True)
df_data_1.head()

,Street,Retail,Office
0,AIRPORT BOULEVARD,326.27,0.00
1,ALBERT STREET,89.32,52.02
2,ALEXANDRA ROAD,109.37,68.89
3,ANG MO KIO AVENUE 3,268.37,0.00
4,ANSON ROAD,82.41,65.91


In [7]:
df_data_1.shape

(152, 3)

In [8]:
# Add a column for Country to get the exact coordinates
df_data_1['Country'] = "Singapore"
df_data_1.head()

,Street,Retail,Office,Country
0,AIRPORT BOULEVARD,326.27,0.00,Singapore
1,ALBERT STREET,89.32,52.02,Singapore
2,ALEXANDRA ROAD,109.37,68.89,Singapore
3,ANG MO KIO AVENUE 3,268.37,0.00,Singapore
4,ANSON ROAD,82.41,65.91,Singapore


In [9]:
df_data_1.shape

(152, 4)

In [10]:
# Find the address of each street location
df_data_1['address']=df_data_1['Street'] + ',' + df_data_1['Country']
df_data_1.head()

,Street,Retail,Office,Country,address
0,AIRPORT BOULEVARD,326.27,0.00,Singapore,"AIRPORT BOULEVARD ,Singapore"
1,ALBERT STREET,89.32,52.02,Singapore,"ALBERT STREET ,Singapore"
2,ALEXANDRA ROAD,109.37,68.89,Singapore,"ALEXANDRA ROAD ,Singapore"
3,ANG MO KIO AVENUE 3,268.37,0.00,Singapore,"ANG MO KIO AVENUE 3 ,Singapore"
4,ANSON ROAD,82.41,65.91,Singapore,"ANSON ROAD ,Singapore"


In [11]:
df_data_1.shape

(152, 5)

In [12]:
# Find the coordinates of each address
nom = Nominatim()
df_data_1['Coordinates'] =df_data_1['address'].apply(nom.geocode)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [13]:
df_data_1.head()

,Street,Retail,Office,Country,address,Coordinates
0,AIRPORT BOULEVARD,326.27,0.00,Singapore,"AIRPORT BOULEVARD ,Singapore","(Airport Boulevard, Changi Business Park, Sout..."
1,ALBERT STREET,89.32,52.02,Singapore,"ALBERT STREET ,Singapore","(Albert Street, Clarke Quay, Selegie, Singapor..."
2,ALEXANDRA ROAD,109.37,68.89,Singapore,"ALEXANDRA ROAD ,Singapore","(Alexandra Road, Alexandra, Sentosa Cove, Sout..."
3,ANG MO KIO AVENUE 3,268.37,0.00,Singapore,"ANG MO KIO AVENUE 3 ,Singapore","(Ang Mo Kio Avenue 3, Serangoon North, Singapo..."
4,ANSON ROAD,82.41,65.91,Singapore,"ANSON ROAD ,Singapore","(Anson Road, Chinatown, Shenton Way, Singapore..."


In [15]:
df_data_1.shape

(152, 6)

#### Find Latitude and Longitude Values

In [16]:
# Find the latitude and longitude values from the coordinates
df_data_1['Latitude']=df_data_1['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_data_1['Longitude']=df_data_1['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_data_1.head()

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude
0,AIRPORT BOULEVARD,326.27,0.00,Singapore,"AIRPORT BOULEVARD ,Singapore","(Airport Boulevard, Changi Business Park, Sout...",1.347311,103.983517
1,ALBERT STREET,89.32,52.02,Singapore,"ALBERT STREET ,Singapore","(Albert Street, Clarke Quay, Selegie, Singapor...",1.303881,103.851074
2,ALEXANDRA ROAD,109.37,68.89,Singapore,"ALEXANDRA ROAD ,Singapore","(Alexandra Road, Alexandra, Sentosa Cove, Sout...",1.278088,103.802975
3,ANG MO KIO AVENUE 3,268.37,0.00,Singapore,"ANG MO KIO AVENUE 3 ,Singapore","(Ang Mo Kio Avenue 3, Serangoon North, Singapo...",1.370656,103.873373
4,ANSON ROAD,82.41,65.91,Singapore,"ANSON ROAD ,Singapore","(Anson Road, Chinatown, Shenton Way, Singapore...",1.275815,103.846491


In [17]:
df_data_1.shape

(152, 8)

In [18]:
# Find the geographical coordinates of Singapore

address = 'Singapore City, SG'
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="sg_explorer", timeout=1000)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore City are 1.340863, 103.830391822121.


### Folium Map

In [19]:
# Create map of Singapoure including all the coordinates
map_sg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, street in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['Street']):
    label = '{}'.format(street)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg) 

In [20]:
map_sg

## Using Foursquare API

In [21]:
CLIENT_ID = 'I2NNHBUHIKCQV3BMQIP1HGHSHC5EZXRGDGRZACS5J0WVN0E1' # your Foursquare ID
CLIENT_SECRET = 'NNPK4VDHHXTOQ2HBS1WZ5QQZN0VW042R5IFDNFFIGWN34SNQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I2NNHBUHIKCQV3BMQIP1HGHSHC5EZXRGDGRZACS5J0WVN0E1
CLIENT_SECRET:NNPK4VDHHXTOQ2HBS1WZ5QQZN0VW042R5IFDNFFIGWN34SNQ


In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [34]:
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
              
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd50d11f7706a001bccd3e8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Yishun',
  'headerFullLocation': 'Yishun, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 1.4397741045000045,
    'lng': 103.84770661212025},
   'sw': {'lat': 1.4307740954999955, 'lng': 103.83872058787975}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d37ca6b9784a0935793dae8',
       'name': 'Zi Zai Vegetarian 自在齋',
       'location': {'address': '236, Yishun Ring Road, #01-1000',
        'lat': 1.4340698931818154,
        'lng': 103.83949549081319,
        'labeledLatLngs': [{'label': 'display',
  

In [36]:
# get the nearby venues
sg_venues = getNearbyVenues(names=df_data_1['Street'], latitudes=df_data_1['Latitude'], longitudes=df_data_1['Longitude'])

AIRPORT BOULEVARD 
ALBERT STREET 
ALEXANDRA ROAD 
ANG MO KIO AVENUE 3 
ANSON ROAD 
BALESTIER ROAD 
BATTERY ROAD
BAYFRONT AVENUE 
BEACH ROAD 
BENCOOLEN STREET 
BROOKE ROAD 
BUKIT TIMAH ROAD
CANTONMENT ROAD
CECIL STREET 
CHANGI BUSINESS PARK CENTRAL 1
CHANGI ROAD 
CHIN SWEE ROAD 
CHOA CHU KANG AVENUE 4 
CHOA CHU KANG NORTH 6 
CHULIA STREET
CHURCH STREET
CLAYMORE DRIVE 
CLEMENCEAU AVENUE 
COLEMAN STREET 
COLLYER QUAY 
COMMONWEALTH AVENUE WEST 
CROSS STREET 
CUPPAGE ROAD 
CUSCADEN ROAD
EAST COAST ROAD 
ENGGOR STREET 
EU TONG SEN STREET
GEYLANG ROAD 
GOLDHILL PLAZA 
GOPENG STREET 
GRANGE ROAD
GUILLEMARD ROAD 
HANDY ROAD 
HARBOUR DRIVE
HARBOURFRONT PLACE 
HAVELOCK ROAD 
HIGH STREET 
HOE CHIANG ROAD 
HOLLAND AVENUE 
HOUGANG AVENUE 10 
HOUGANG STREET 51 
HOUGANG STREET 91 
IRRAWADDY ROAD 
JALAN ANAK BUKIT 
JALAN BESAR 
JALAN JURONG KECHIL 
JALAN MASJID 
JALAN SULTAN 
JELEBU ROAD 
JOO CHIAT ROAD
JURONG EAST STREET 21
JURONG GATEWAY ROAD 
JURONG WEST CENTRAL 2
JURONG WEST CENTRAL 3 
KEPPEL ROAD 

In [37]:
sg_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIRPORT BOULEVARD,1.347311,103.983517,World of Sports 3-Top T3,1.347745,103.984680,Sporting Goods Shop
1,AIRPORT BOULEVARD,1.347311,103.983517,Airport Boulevard,1.345959,103.983836,Road
2,AIRPORT BOULEVARD,1.347311,103.983517,Dnata Inflight Catering (CIAS),1.347647,103.985100,Airport Terminal
3,AIRPORT BOULEVARD,1.347311,103.983517,Hugo Boss,1.347255,103.985423,Men's Store
4,AIRPORT BOULEVARD,1.347311,103.983517,Pizza Hut T3,1.349500,103.983649,Pizza Place


In [38]:
sg_venues.shape

(8503, 7)

In [39]:
print(sg_venues.shape)
sg_venues

(8503, 7)


,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIRPORT BOULEVARD,1.347311,103.983517,World of Sports 3-Top T3,1.347745,103.984680,Sporting Goods Shop
1,AIRPORT BOULEVARD,1.347311,103.983517,Airport Boulevard,1.345959,103.983836,Road
2,AIRPORT BOULEVARD,1.347311,103.983517,Dnata Inflight Catering (CIAS),1.347647,103.985100,Airport Terminal
3,AIRPORT BOULEVARD,1.347311,103.983517,Hugo Boss,1.347255,103.985423,Men's Store
4,AIRPORT BOULEVARD,1.347311,103.983517,Pizza Hut T3,1.349500,103.983649,Pizza Place
5,AIRPORT BOULEVARD,1.347311,103.983517,Cabin Crew Taxi ( T2 ),1.344809,103.983371,Airport Terminal
6,AIRPORT BOULEVARD,1.347311,103.983517,Cebu Pacific Airways (5J) Check-In Counter,1.348497,103.985728,Airport
7,AIRPORT BOULEVARD,1.347311,103.983517,7D cinema,1.349036,103.981671,Movie Theater
8,AIRPORT BOULEVARD,1.347311,103.983517,SATS Inflight Catering Centre 1 Canteen,1.344195,103.983178,Café
9,AIRPORT BOULEVARD,1.347311,103.983517,So Chocolate,1.350596,103.983807,Candy Store


In [40]:
print (sg_venues['Venue Category'].unique())

['Sporting Goods Shop' 'Road' 'Airport Terminal' "Men's Store"
 'Pizza Place' 'Airport' 'Movie Theater' 'Café' 'Candy Store'
 'Airport Gate' 'Gaming Cafe' 'Indian Restaurant' 'Arts & Crafts Store'
 'Vegetarian / Vegan Restaurant' 'Chinese Restaurant' 'BBQ Joint'
 'General College & University' 'Art Gallery' 'Asian Restaurant'
 'Ice Cream Shop' 'Hostel' 'Middle Eastern Restaurant' 'Music Venue'
 'Bakery' 'Dive Bar' 'Hotel' 'Food Court' 'Japanese Restaurant'
 'Coffee Shop' 'Food' 'Garden' 'Bridge' 'Bar' 'Seafood Restaurant'
 'Cocktail Bar' 'Salad Place' 'Electronics Store' 'Mexican Restaurant'
 'Fast Food Restaurant' 'French Restaurant' 'American Restaurant'
 'Tech Startup' 'Fried Chicken Joint' 'Wine Bar' 'Sushi Restaurant'
 'Deli / Bodega' 'Restaurant' 'Soup Place' 'Sandwich Place' 'Bus Station'
 'Shopping Mall' 'Supermarket' 'Italian Restaurant' 'Breakfast Spot'
 'Pet Store' 'Noodle House' 'Skate Park' 'Badminton Court'
 'Gym / Fitness Center' 'Ramen Restaurant' 'Bagel Shop'
 'Spanish

In [41]:
# According to use case for the street location for office or retail space, we only require Bus Station, Train Station and Food Court as Venue Categories
sg_venues=sg_venues.loc[(sg_venues['Venue Category'] == 'Food Court') | (sg_venues['Venue Category'] == 'Train Station') | (sg_venues['Venue Category'] == 'Bus Station')]

In [42]:
sg_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
42,ALBERT STREET,1.303881,103.851074,Albert Centre Market & Food Centre,1.301024,103.854477,Food Court
80,ALEXANDRA ROAD,1.278088,103.802975,Bus Stop 18019,1.281130,103.802311,Bus Station
82,ALEXANDRA ROAD,1.278088,103.802975,Bus Stop 14369 (Gilman Hts Condo,1.281373,103.803256,Bus Station
94,ANG MO KIO AVENUE 3,1.370656,103.873373,Ghee Hong Eating House,1.369535,103.871813,Food Court
96,ANG MO KIO AVENUE 3,1.370656,103.873373,Cafe 107,1.369715,103.869961,Food Court


In [43]:
sg_venues.shape

(237, 7)

### Get categorical values

In [44]:
# one hot encoding
sg_onehot = pd.get_dummies(sg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['Street'] = sg_venues['Street'] 

# move neighborhood column to the first column
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

sg_onehot.head()

,Street,Bus Station,Food Court,Train Station
42,ALBERT STREET,0,1,0
80,ALEXANDRA ROAD,1,0,0
82,ALEXANDRA ROAD,1,0,0
94,ANG MO KIO AVENUE 3,0,1,0
96,ANG MO KIO AVENUE 3,0,1,0


In [45]:
sg_onehot.shape

(237, 4)

In [47]:
sg_grouped = sg_onehot.groupby('Street').mean().reset_index()
sg_grouped.head()

,Street,Bus Station,Food Court,Train Station
0,ALBERT STREET,0.000000,1.000000,0.0
1,ALEXANDRA ROAD,1.000000,0.000000,0.0
2,ANG MO KIO AVENUE 3,0.333333,0.666667,0.0
3,ANSON ROAD,0.000000,1.000000,0.0
4,BALESTIER ROAD,0.500000,0.500000,0.0


In [48]:
sg_grouped.shape

(111, 4)

In [49]:
# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
# Now let's create the new dataframe and display the top 3 venues for each neighborhood.
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
street_venues_sorted = pd.DataFrame(columns=columns)
street_venues_sorted['Street'] = sg_grouped['Street']

for ind in np.arange(sg_grouped.shape[0]):
    street_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sg_grouped.iloc[ind, :], num_top_venues)

street_venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,ALBERT STREET,Food Court,Train Station,Bus Station
1,ALEXANDRA ROAD,Bus Station,Train Station,Food Court
2,ANG MO KIO AVENUE 3,Food Court,Bus Station,Train Station
3,ANSON ROAD,Food Court,Train Station,Bus Station
4,BALESTIER ROAD,Food Court,Bus Station,Train Station


In [52]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

sg_grouped_clustering = sg_grouped.drop('Street', 1)

sg_grouped_clustering['Office'] = df_data_1['Office']
sg_grouped_clustering['Retail'] = df_data_1['Retail']

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 7, 4, 2, 7, 7, 3, 5, 7, 4], dtype=int32)

In [53]:
sg_grouped_clustering.head()

,Bus Station,Food Court,Train Station,Office,Retail
0,0.000000,1.000000,0.0,0.00,326.27
1,1.000000,0.000000,0.0,52.02,89.32
2,0.333333,0.666667,0.0,68.89,109.37
3,0.000000,1.000000,0.0,0.00,268.37
4,0.500000,0.500000,0.0,65.91,82.41


In [54]:
sg_grouped_clustering.shape

(111, 5)

In [59]:
# add clustering labels
street_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sg_merged = df_data_1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sg_merged = sg_merged.join(street_venues_sorted.set_index('Street'), on='Street')

In [60]:
sg_merged.head()

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,AIRPORT BOULEVARD,326.27,0.00,Singapore,"AIRPORT BOULEVARD ,Singapore","(Airport Boulevard, Changi Business Park, Sout...",1.347311,103.983517,NaN,NaN,NaN,NaN
1,ALBERT STREET,89.32,52.02,Singapore,"ALBERT STREET ,Singapore","(Albert Street, Clarke Quay, Selegie, Singapor...",1.303881,103.851074,5.0,Food Court,Train Station,Bus Station
2,ALEXANDRA ROAD,109.37,68.89,Singapore,"ALEXANDRA ROAD ,Singapore","(Alexandra Road, Alexandra, Sentosa Cove, Sout...",1.278088,103.802975,7.0,Bus Station,Train Station,Food Court
3,ANG MO KIO AVENUE 3,268.37,0.00,Singapore,"ANG MO KIO AVENUE 3 ,Singapore","(Ang Mo Kio Avenue 3, Serangoon North, Singapo...",1.370656,103.873373,4.0,Food Court,Bus Station,Train Station
4,ANSON ROAD,82.41,65.91,Singapore,"ANSON ROAD ,Singapore","(Anson Road, Chinatown, Shenton Way, Singapore...",1.275815,103.846491,2.0,Food Court,Train Station,Bus Station


In [61]:
sg_merged.tail()

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
147,WOODLANDS AVENUE 1,72.58,0.0,Singapore,"WOODLANDS AVENUE 1 ,Singapore","(Woodlands Avenue 1, Woodlands, Northwest, 737...",1.432085,103.787597,1.0,Food Court,Bus Station,Train Station
148,WOODLANDS ROAD,66.10,0.0,Singapore,"WOODLANDS ROAD ,Singapore","(Woodlands Road, Zhenghua, Northwest, 688258, ...",1.386184,103.756610,NaN,NaN,NaN,NaN
149,YIO CHU KANG ROAD,47.83,0.0,Singapore,"YIO CHU KANG ROAD ,Singapore","(Yio Chu Kang Road, Mei Hwan, Southeast, 54566...",1.367040,103.875386,0.0,Food Court,Train Station,Bus Station
150,YISHUN AVENUE 2,229.99,0.0,Singapore,"YISHUN AVENUE 2 ,Singapore","(Yishun Avenue 2, Singapore, Central, 769139, ...",1.411064,103.830264,NaN,NaN,NaN,NaN
151,YISHUN AVENUE 9,87.15,0.0,Singapore,"YISHUN AVENUE 9 ,Singapore","(Yishun Avenue 9, Yishun, Northwest, 761317, S...",1.435274,103.843214,NaN,NaN,NaN,NaN


In [62]:
sg_merged['Cluster Labels'].fillna(0, inplace = True)
sg_merged.head()       

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,AIRPORT BOULEVARD,326.27,0.00,Singapore,"AIRPORT BOULEVARD ,Singapore","(Airport Boulevard, Changi Business Park, Sout...",1.347311,103.983517,0.0,NaN,NaN,NaN
1,ALBERT STREET,89.32,52.02,Singapore,"ALBERT STREET ,Singapore","(Albert Street, Clarke Quay, Selegie, Singapor...",1.303881,103.851074,5.0,Food Court,Train Station,Bus Station
2,ALEXANDRA ROAD,109.37,68.89,Singapore,"ALEXANDRA ROAD ,Singapore","(Alexandra Road, Alexandra, Sentosa Cove, Sout...",1.278088,103.802975,7.0,Bus Station,Train Station,Food Court
3,ANG MO KIO AVENUE 3,268.37,0.00,Singapore,"ANG MO KIO AVENUE 3 ,Singapore","(Ang Mo Kio Avenue 3, Serangoon North, Singapo...",1.370656,103.873373,4.0,Food Court,Bus Station,Train Station
4,ANSON ROAD,82.41,65.91,Singapore,"ANSON ROAD ,Singapore","(Anson Road, Chinatown, Shenton Way, Singapore...",1.275815,103.846491,2.0,Food Court,Train Station,Bus Station


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['Street'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

In [66]:
map_clusters

### Cluster Label = 1

In [67]:
sg_1=sg_merged.loc[(sg_merged['Cluster Labels'] == 1) & (sg_merged['Retail'] < 200) & (sg_merged['Retail'] != 0)]
sg_1.head()

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
15,CHANGI ROAD,58.41,38.89,Singapore,"CHANGI ROAD ,Singapore","(Changi Road, Joo Chiat, Southeast, 419871, Si...",1.318717,103.910811,1.0,Food Court,Train Station,Bus Station
27,CUPPAGE ROAD,74.53,0.00,Singapore,"CUPPAGE ROAD ,Singapore","(Cuppage Road, Morison Hill, Orchard, Singapor...",1.302461,103.840581,1.0,Food Court,Train Station,Bus Station
35,GRANGE ROAD,87.16,0.00,Singapore,"GRANGE ROAD,Singapore","(Grange Road, Morison Hill, Orchard, Singapore...",1.299824,103.831894,1.0,Bus Station,Train Station,Food Court
36,GUILLEMARD ROAD,59.87,0.00,Singapore,"GUILLEMARD ROAD ,Singapore","(Guillemard Road, Geylang, Mei Hwan, Southeast...",1.310651,103.882750,1.0,Food Court,Train Station,Bus Station
37,HANDY ROAD,68.73,0.00,Singapore,"HANDY ROAD ,Singapore","(Handy Road, Robertson Quay, Selegie, Singapor...",1.299894,103.846127,1.0,Food Court,Train Station,Bus Station


In [69]:
# create map
map_1 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_1['Latitude'], sg_1['Longitude'], sg_1['Street'], sg_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_1)
       
map_1

### Cluster Label = 2

In [70]:
sg_2=sg_merged.loc[(sg_merged['Cluster Labels'] == 2) & (sg_merged['Office'] < 200) & (sg_merged['Office'] != 0)]
sg_2.head()

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
4,ANSON ROAD,82.41,65.91,Singapore,"ANSON ROAD ,Singapore","(Anson Road, Chinatown, Shenton Way, Singapore...",1.275815,103.846491,2.0,Food Court,Train Station,Bus Station
33,GOLDHILL PLAZA,43.61,48.32,Singapore,"GOLDHILL PLAZA ,Singapore","(Goldhill Plaza, Newton Circus, Singapore, Cen...",1.317911,103.843807,2.0,Bus Station,Train Station,Food Court
67,MALACCA STREET,0.00,87.50,Singapore,"MALACCA STREET,Singapore","(Malacca Street, Chinatown, Raffles Place, Sin...",1.283720,103.851239,2.0,Food Court,Train Station,Bus Station
72,MARITIME SQUARE,179.46,73.72,Singapore,"MARITIME SQUARE ,Singapore","(Maritime Square, Radin Mas, Harbourfront, Sin...",1.264222,103.818926,2.0,Food Court,Train Station,Bus Station
73,MARKET STREET,0.00,104.24,Singapore,"MARKET STREET ,Singapore","(Market Street, Chinatown, Raffles Place, Sing...",1.282425,103.850253,2.0,Food Court,Train Station,Bus Station


In [72]:
# create map
map_2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_2['Latitude'], sg_2['Longitude'], sg_2['Street'], sg_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_2)
       
map_2

### Cluster Label = 0

In [73]:
sg_0=sg_merged.loc[(sg_merged['Cluster Labels'] == 0) & (sg_merged['Office'] < 200) & (sg_merged['Office'] != 0)]
sg_0.head()

,Street,Retail,Office,Country,address,Coordinates,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
6,BATTERY ROAD,0.00,139.38,Singapore,"BATTERY ROAD,Singapore","(Battery Road, Chinatown, Raffles Place, Singa...",1.285246,103.852068,0.0,NaN,NaN,NaN
11,BUKIT TIMAH ROAD,102.36,58.11,Singapore,"BUKIT TIMAH ROAD,Singapore","(Bukit Timah Road, Novena, Singapore, Central,...",1.320578,103.823761,0.0,NaN,NaN,NaN
19,CHULIA STREET,0.00,72.03,Singapore,"CHULIA STREET,Singapore","(Chulia Street, Chinatown, Raffles Place, Sing...",1.285500,103.849477,0.0,Food Court,Train Station,Bus Station
23,COLEMAN STREET,70.21,59.43,Singapore,"COLEMAN STREET ,Singapore","(Coleman Street, Clarke Quay, City Hall, Singa...",1.291057,103.852481,0.0,NaN,NaN,NaN
24,COLLYER QUAY,204.77,121.30,Singapore,"COLLYER QUAY ,Singapore","(Collyer Quay, Chinatown, Raffles Place, Singa...",1.283807,103.852909,0.0,Food Court,Train Station,Bus Station


In [75]:
# create map
map_0 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_0['Latitude'], sg_0['Longitude'], sg_0['Street'], sg_0['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_0)
       
map_0